In [12]:
import sys
sys.path.append('../')
import pandas as pd
import ta
from Tools.backtest_tools import get_n_columns, get_metrics, basic_single_asset_backtest, plot_wallet_vs_asset
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [80]:
class BBands:
    def __init__(
            self, 
            df, 
            type=['long', 'short'],
            BB_window = 100,
            BB_std = 2.25,
            max_BB_speard = 0,
            ma_window = 20) -> None:
        
        self.df = df
        self.use_long = True if "long" in type else False
        self.use_short = True if "short" in type else False
        self.BB_window = BB_window
        self.BB_std = BB_std
        self.max_BB_spread = max_BB_speard
        self.ma_window = ma_window

    def populate_indicators(self):
        # -- Clear dataset --
        df = self.df
        df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
        for i in range(1,4):
            df[f"open_long_market_{i}"] = False
            df[f"close_long_market_{i}"] = False
            df[f"open_short_market_{i}"] = False
            df[f"close_short_market_{i}"] = False
        
        # -- Populate indicators --
        mult1, mult2, mult3 = 1, 1.5, 2
        df['ma_band'] = df['close'].rolling(window=self.ma_window).mean()
        dev =  df['close'].rolling(window=100).std()
        df['lvl1_above'] = df['ma_band'] + dev * mult1
        df['lvl2_above'] = df['ma_band'] + dev * mult2
        df['lvl3_above'] = df['ma_band'] + dev * mult3
        df['lvl1_below'] = df['ma_band'] - dev * mult1
        df['lvl2_below'] = df['ma_band'] - dev * mult2
        df['lvl3_below'] = df['ma_band'] - dev * mult3

        df = get_n_columns(df, ["lvl1_above", "lvl2_above", "lvl3_above", "lvl1_below", "lvl2_below", "lvl3_below", "close"], 1)
        
        self.df = df    
        return self.df
 
    def populate_buy_sell(self): 
        df = self.df
        # -- Initiate populate --
        tracker = dict(
        lvl2_above = False,
        lvl3_above = False,
        lvl1_above = False,
        lvl1_below = False,
        lvl2_below = False,
        lvl3_below = False)

        
        if self.use_long:
            # -- Populate open long market --
            for i in range(1,4):
                df.loc[
                    (df['n1_close'] < df[f'lvl{i}_below']) 
                    & (df['close'] > df[f'lvl{i}_below']) 
                    & ((df['lvl3_above'] - df['lvl3_below']) / df['ma_band'] > self.max_BB_spread)
                    & (df["close"] < df["ma_band"]) 
                    & (~tracker[f'lvl{i}_below'])
                    , f"open_long_market_{i}"
                ] = True
                if df[f'open_long_market_{i}'].all(): tracker[f'lvl{i}_below'] = True
                # -- Populate close long market --
                df.loc[
                    (df['close'] > df['ma_band']) 
                    , f"close_long_market_{i}"
                ] = True
                tracker["lvl1_below"] = False
                tracker["lvl2_below"] = False
                tracker["lvl3_below"] = False
        if self.use_short:
            # -- Populate open short market --
            for i in range(1,4):
                df.loc[
                    (df['n1_close'] > df[f'lvl{i}_above']) 
                    & (df['close'] < df[f'lvl{i}_above']) 
                    & ((df['lvl3_above'] - df['lvl3_below']) / df['ma_band'] > self.max_BB_spread)
                    & (df["close"] > df["ma_band"])
                    & (~tracker[f'lvl{i}_above'])
                    , f"open_short_market_{i}"
                ] = True
                if df[f'open_short_market_{i}'].all(): tracker[f'lvl{i}_above'] = True 
                # -- Populate close short market --
                df.loc[
                    (df['close'] < df['ma_band']) 
                    , f"close_short_market_{i}"
                ] = True
                tracker["lvl3_above"] = False
                tracker["lvl2_above"] = False
                tracker["lvl1_above"] = False
        self.df = df   
        return self.df
    
    def run_backtest(self, initial_wallet=1000, leverage=1):
        # Initialize variables
        df = self.df
        wallet = initial_wallet
        maker_fee = 0.0002
        taker_fee = 0.0007
        trades = []
        days = []
        current_day = 0
        previous_day = 0
        current_position_1 = None
        current_position_2 = None
        current_position_3 = None

        # Iterate over dataframe rows
        for index, row in df.iterrows():
            # Check for closing position
            for i in range(1, 4):
                current_position = locals()[f'current_position_{i}']
                if current_position is not None:
                    if current_position['side'] == 'LONG':
                        if row[f'close_long_market_{i}']:
                            close_price = row['close']
                            trade_result = ((close_price - current_position['price']) / current_position['price']) * leverage
                            wallet += wallet * trade_result
                            fee = wallet * taker_fee
                            wallet -= fee
                            trades.append({
                                'open_date': current_position['date'],
                                'close_date': index,
                                'position': 'LONG',
                                'open_reason': current_position['reason'],
                                'close_reason': 'Market',
                                'open_price': current_position['price'],
                                'close_price': close_price,
                                'open_fee': current_position['fee'],
                                'close_fee': fee,
                                'open_trade_size': current_position['size'],
                                'close_trade_size': wallet,
                                'wallet': wallet
                            })
                            locals()[f'current_position_{i}'] = None
                    elif current_position['side'] == 'SHORT':
                        if row[f'close_short_market_{i}']:
                            close_price = row['close']
                            trade_result = ((current_position['price'] - close_price) / current_position['price']) * leverage
                            wallet += wallet * trade_result
                            fee = wallet * taker_fee
                            wallet -= fee
                            trades.append({
                                'open_date': current_position['date'],
                                'close_date': index,
                                'position': 'SHORT',
                                'open_reason': current_position['reason'],
                                'close_reason': 'Market',
                                'open_price': current_position['price'],
                                'close_price': close_price,
                                'open_fee': current_position['fee'],
                                'close_fee': fee,
                                'open_trade_size': current_position['size'],
                                'close_trade_size': wallet,
                                'wallet': wallet
                            })
                            locals()[f'current_position_{i}'] = None

            # Check for opening position
            for i in range(1, 4):
                if locals()[f'current_position_{i}'] is None:
                    if row[f'open_long_market_{i}']:
                        open_price = row['close']
                        fee = wallet * taker_fee
                        wallet -= fee
                        pos_size = wallet
                        locals()[f'current_position_{i}'] = {
                            'size': pos_size,
                            'date': index,
                            'price': open_price,
                            'fee': fee,
                            'reason': 'Market',
                            'side': 'LONG'
                        }
                    elif row[f'open_short_market_{i}']:
                        open_price = row['close']
                        fee = wallet * taker_fee
                        wallet -= fee
                        pos_size = wallet
                        locals()[f'current_position_{i}'] = {
                            'size': pos_size,
                            'date': index,
                            'price': open_price,
                            'fee': fee,
                            'reason': 'Market',
                            'side': 'SHORT'
                        }

        # for index, row in df.iterrows():
        #     #-- Add daily report --
        #     current_day = index.day
        #     if previous_day != current_day:
        #         temp_wallet = wallet
        #         for i in range(1, 4):
        #             current_position = locals()[f'current_position_{i}']
        #             if current_position is not None:
        #                 if current_position['side'] == 'LONG':
        #                     close_price = row['close']
        #                     trade_result = (close_price - current_position['price']) / current_position['price']
        #                     temp_wallet += temp_wallet * trade_result
        #                     fee = temp_wallet * taker_fee
        #                     temp_wallet -= fee
        #                 elif current_position['side'] == 'SHORT':
        #                     close_price = row['close']
        #                     trade_result = (current_position['price'] - close_price) / current_position['price']
        #                     temp_wallet += temp_wallet * trade_result
        #                     fee = temp_wallet * taker_fee
        #                     temp_wallet -= fee
        #         days.append({
        #             'day': index.date(),
        #             'wallet': temp_wallet,
        #             'price': row['close']
        #             })
        #     previous_day = current_day

        # # Convert lists to dataframes
        # df_days = pd.DataFrame(days)
        # df_days = df_days.set_index('day')

        # df_trades = pd.DataFrame(trades)
        # if df_trades.empty:
        #     print('!!! No trades')
        #     return None
        # else:
        #     df_trades = df_trades.set_index('open_date')

        # # Calculate metrics
        # metrics = get_metrics(df_trades, df_days)
        # metrics['wallet'] = wallet
        # metrics['trades'] = df_trades
        # metrics['days'] = df_days

        # return metrics
        return trades
    
    def plot(self):
        df = self.df
        plt.figure(figsize=(12, 6))
        plt.plot(df['close'], label='Close Price')
        plt.plot(df['ma_band'], label='Moving Average')
        plt.plot(df['lvl1_above'], label='BB Upper 1')
        plt.plot(df['lvl2_above'], label='BB Upper 2')
        plt.plot(df['lvl3_above'], label='BB Upper 3')
        plt.plot(df['lvl1_below'], label='BB Lower 1')
        plt.plot(df['lvl2_below'], label='BB Lower 2')
        plt.plot(df['lvl3_below'], label='BB Lower 3')
        plt.legend()
        plt.show()
    
      

In [73]:
df = pd.read_csv('../DB/crvusdt.csv')
df['datetime'] = pd.to_datetime(df['datetime'], unit="ms")
df.set_index('datetime', inplace=True)

In [15]:
df.dtypes

open      float64
high      float64
low       float64
close     float64
volume    float64
dtype: object

In [83]:
strat = BBands(
    df = df,
    type=["long","short"],
    BB_window = 100,
    BB_std = 1000,
    max_BB_speard = 0,
    ma_window = 20,
)

strat.populate_indicators()
buy_sell = strat.populate_buy_sell()
trades = strat.run_backtest()

# trades

In [ ]:
# strat.populate_buy_sell()
# bt_result = strat.run_backtest(initial_wallet=1000, leverage=1)

In [ ]:
bt_result = strat.run_backtest(initial_wallet=1000, leverage=1)
df_trades, df_days = basic_single_asset_backtest(trades=bt_result['trades'], days=bt_result['days'])
plot_wallet_vs_asset(df_days=df_days)